In [3]:
import pandas as pd


In [1]:
df = pd.read_csv('/home/maali/code/3Maali/VAK_Project/raw_data/dataset.csv')

NameError: name 'pd' is not defined

In [ ]:
df.head()

In [ ]:
df.info()

In [ ]:
df.describe()

In [ ]:
df.isnull().sum()

In [ ]:
# التأكد من أن التصنيفات في عمود Type هي: Auditory، Visual، Kinesthetic فقط.
df_types = df['Type'].unique()
# معرفة توزيع الأنماط (عدد الجمل لكل نمط).
print(df_types)
df['Type'].value_counts()

In [ ]:

# # التأكد من أن جميع القيم في عمود Type صحيحة
# valid_types = ['Auditory', 'Visual', 'Kinesthetic']
# dataset = dataset[dataset['Type'].isin(valid_types)]

# عرض عدد الصفوف بعد التنظيف
print(f"\nعدد الصفوف بعد التنظيف: {len(df)}")

### Text Preprocessing


In [ ]:
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
import string

In [ ]:
# تحميل الموارد اللازمة من NLTK
nltk.download('punkt')
nltk.download('stopwords')
nltk.download('wordnet')

In [ ]:
# إعداد قائمة الكلمات الشائعة وعلامات الترقيم
stop_words = set(stopwords.words('english'))
lemmatizer = WordNetLemmatizer()
stop_words

In [ ]:
# دالة لمعالجة النصوص
def preprocess_text(text):
    # تحويل إلى حروف صغيرة
    text = text.lower()
    # إزالة علامات الترقيم
    text = text.translate(str.maketrans('', '', string.punctuation))
    # تقطيع النص إلى كلمات
    words = word_tokenize(text)
    # إزالة الكلمات الشائعة وتوحيد الكلمات
    words = [lemmatizer.lemmatize(word) for word in words if word not in stop_words]
    # إعادة الكلمات كنص واحد
    return ' '.join(words)

In [ ]:

df['Processed_Sentence'] = df['Sentence'].apply(preprocess_text)

In [ ]:
df

الخطوة 4: تحويل النصوص إلى تمثيلات رقمية
لاستخدام النصوص في نماذج التعلم الآلي، نحتاج إلى تحويلها إلى تمثيلات رقمية. سنستخدم TF-IDF كمثال، لأنها طريقة شائعة وفعالة لتصنيف النصوص.


In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.preprocessing import LabelEncoder

In [ ]:
# إنشاء متجه TF-IDF
vectorizer = TfidfVectorizer(max_features=1000)  # تحديد عدد الميزات إلى 1000 كمثال
X = vectorizer.fit_transform(df['Processed_Sentence'])

In [ ]:
# تحويل الأنماط إلى أرقام (Label Encoding)

label_encoder = LabelEncoder()
y = label_encoder.fit_transform(df['Type'])

In [ ]:
# عرض شكل المصفوفة الناتجة
print(f"شكل مصفوفة TF-IDF: {X.shape}")
print(f"التصنيفات المشفرة: {y[:5]}")
print(f"الأنماط الأصلية: {label_encoder.classes_}")

TfidfVectorizer: يحول النصوص إلى مصفوفة TF-IDF، حيث يتم وزن الكلمات بناءً على أهميتها.

max_features=1000: يحدد عدد الكلمات الأكثر أهمية لتقليل الأبعاد.

LabelEncoder: يحول التصنيفات (Auditory، Visual، Kinesthetic) إلى أرقام (مثل 0، 1، 2).

X.shape: يعرض أبعاد المصفوفة الناتجة (عدد الجمل × عدد الميزات).

label_encoder.classes_: يعرض الأنماط الأصلية المقابلة للأرقام.


الآن البيانات جاهزة لتدريب نموذج تصنيف (مثل Logistic Regression أو Random Forest).




### استكشاف البيانات (Exploratory Data Analysis)


الهدف هنا هو فهم البيانات بشكل أفضل من خلال:
توزيع الأنماط: معرفة عدد الجمل لكل نمط (Auditory، Visual، Kinesthetic).

الكلمات الشائعة: تحديد الكلمات الأكثر تكرارًا لكل نمط باستخدام Word Clouds.

طول الجمل: تحليل طول الجمل لمعرفة ما إذا كان يختلف بين الأنماط.


In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns
from wordcloud import WordCloud
from collections import Counter

In [ ]:
# 1. توزيع الأنماط
plt.figure(figsize=(8, 6))
sns.countplot(data=df, x='Type')
plt.title('توزيع أنماط التعلم')
plt.xlabel('النمط')
plt.ylabel('عدد الجمل')
plt.show()

In [ ]:

import matplotlib.pyplot as plt
from wordcloud import WordCloud

# دالة لإنشاء Word Cloud
def plot_wordcloud(text, title):
    # تحديد مسار الخط DejaVuSans.ttf
    font_path = '/usr/share/fonts/truetype/dejavu/DejaVuSans.ttf'

    # إنشاء WordCloud مع تحديد font_path
    wordcloud = WordCloud(width=800, height=400, background_color='white', font_path=font_path).generate(text)
    plt.figure(figsize=(10, 5))
    plt.imshow(wordcloud, interpolation='bilinear')
    plt.title(title)
    plt.axis('off')
    plt.show()

# تجميع النصوص لكل نمط وإنشاء Word Clouds
for learning_style in df['Type'].unique():
    style_text = ' '.join(df[df['Type'] == learning_style]['Processed_Sentence'])
    plot_wordcloud(style_text, f'Word Cloud لنمط {learning_style}')

In [ ]:
from PIL import ImageFont
font_path = '/usr/share/fonts/truetype/liberation/LiberationSans-Regular.ttf'
try:
    font = ImageFont.truetype(font_path, size=12)
    print("الخط صالح!")
except Exception as e:
    print(f"خطأ في الخط: {e}")

In [ ]:
# 3. تحليل طول الجمل
df['Sentence_Length'] = df['Processed_Sentence'].apply(lambda x: len(x.split()))
plt.figure(figsize=(8, 6))
sns.boxplot(data=df, x='Type', y='Sentence_Length')
plt.title('توزيع طول الجمل حسب النمط')
plt.xlabel('النمط')
plt.ylabel('عدد الكلمات في الجملة')
plt.show()

توزيع الأنماط: نستخدم countplot لرسم مخطط أعمدة يوضح عدد الجمل لكل نمط.

Word Clouds: نستخدم مكتبة wordcloud لإنشاء سحابة كلمات تعرض الكلمات الأكثر تكرارًا لكل نمط.

طول الجمل: نحسب عدد الكلمات في كل جملة (بعد المعالجة) ونرسم Box Plot لمقارنة طول الجمل بين الأنماط.

_--___--_--
تساعدنا هذه التحليلات على فهم السمات اللغوية لكل نمط.

إذا كان هناك خلل في التوزيع (مثل نمط واحد يهيمن)، قد نحتاج إلى موازنة البيانات (مثل Oversampling أو Undersampling).

